In [6]:
import torch, torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as tf
import numpy as np
from matplotlib import pyplot as plt

# dataset to be finalised
# assumption -> image is 28x28, input vector size = 784 -> to be changed depending on the final dataset

# Note : to be filled
# batch_size = 
# train =
# train_loader =
# test =  
# test_loader =

# initialize random seeds; select gpu device if available
torch.manual_seed(1)
torch.cuda.manual_seed(1) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# function to display the reconstructed images
# this function can be called during training/testing
def display_images(in_, out, n=1, label=None, count=False):
    for N in range(n):
        if in_ is not None:
            in_pic = in_.data.cpu().view(-1, 28, 28)
            plt.figure(figsize=(20, 4))
            plt.suptitle(label + ' – reconstructed images', color='w', fontsize=20)
            for i in range(4):
                plt.subplot(1,4,i+1)
                plt.imshow(in_pic[i+4*N])
                plt.axis('off')
        out_pic = out.data.cpu().view(-1, 28, 28)
        plt.figure(figsize=(20, 6))
        for i in range(4):  
            plt.subplot(1,4,i+1)
            plt.imshow(out_pic[i+4*N])
            plt.axis('off')
            if count: plt.title(str(4 * N + i), color='w')

In [ ]:
# creating basic VAE model for image reconstruction; depth encoding - pending

d = 10 # data points can be varied accordingly
class VAE(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(nn.Linear(784, d ** 2),nn.ReLU(),nn.Linear(d ** 2, d * 2))
        self.decoder = nn.Sequential(nn.Linear(d, d ** 2),nn.ReLU(),nn.Linear(d ** 2, 784),nn.Sigmoid(),)
        
        # Note : classes for Encoder/Decoder can be added to encode depths using convolutions
        # class Encoder(nn.Module):
        # class Decoder(nn.Module):

    def reparam(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()  # compute standard dev using non-negative logvariance
            eps = std.data.new(std.size()).normal_() # to get normal distribution for backprop in training
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x):
        mu_logvar = self.encoder(x.view(-1, 784)).view(-1, 2, d)
        mu = mu_logvar[:, 0, :]
        logvar = mu_logvar[:, 1, :]  # calculating the value of 'mu' and 'logvar' from encoder
        z = self.reparam(mu, logvar) # finding z by reparameterization
        return self.decoder(z), mu, logvar # return the decoder output

model = VAE().to(device)

# Setting the optimiser
optimizer = torch.optim.Adam(model.parameters(),lr=0.01) # learning rate can be modified accordingly

# define the loss function -> using binary_cross_entropy loss with KL divergence
def loss_function(x_hat, x, mu, logvar):  # where x = instance of training; x_hat, mu, logvar = model(x); same concept can be applied for testing phase
    BCE = nn.functional.binary_cross_entropy(x_hat, x.view(-1, 784), reduction='sum')
    KLDiv = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))
    return BCE + KLDiv

#write code to train and test the model